In [1]:
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt

In [ ]:
def 

## Скачиваем журнал

In [2]:
file_name = 'ИК Папки 2020г.xls'
sheet_name = 'ИК 2020 Р'

In [3]:
xldf = pd.read_excel(file_name, sheet_name=sheet_name,
                    header=1, nrows=170)

# убираем безымянные стобцы
xldf.drop(list(filter(lambda x: re.search(r'^Unnamed:', x) is not None,
                      xldf.columns)), axis=1, inplace=True)

# убираем пробелы по краям названий стобов
xldf.columns = list(map(lambda x: re.sub(r'\s+', ' ', x.strip()).lower().capitalize(), xldf.columns))
xldf.dtypes

№ п/п                       float64
Дата                         object
№ договора                   object
Наименование организации     object
Адрес заказчика              object
Вид работ                    object
Стоимость работ              object
Место хранения               object
Исполнитель                  object
Основание                    object
Срок исполнения              object
Срок действия договора       object
№ акта                       object
Число                        object
№ протокола                  object
№ заявки                     object
dtype: object

## Преобразование типов

In [4]:
xldf['Стоимость работ'] =\
xldf['Стоимость работ'].apply(lambda x: re.sub(r'\s', '', x).replace(',', '.')
                             if type(x) == str
                             else x)
xldf.dtypes

№ п/п                       float64
Дата                         object
№ договора                   object
Наименование организации     object
Адрес заказчика              object
Вид работ                    object
Стоимость работ              object
Место хранения               object
Исполнитель                  object
Основание                    object
Срок исполнения              object
Срок действия договора       object
№ акта                       object
Число                        object
№ протокола                  object
№ заявки                     object
dtype: object

In [5]:
xldf = xldf.astype({
    '№ п/п':                                'Int64',
    'Дата':                            'datetime64',
    '№ договора':                          'string',
    'Наименование организации':            'string',
    'Адрес заказчика':                     'string',
    'Вид работ':                           'string',
    'Стоимость работ':                    'float64',
    'Место хранения':                      'string',
    'Исполнитель':                         'string',
    'Основание':                           'string',
    'Срок исполнения':                     'string',
    'Срок действия договора':              'string',
    '№ акта':                              'string',
    'Число':                           'datetime64',
    '№ протокола':                         'string',
    '№ заявки':                            'string',
})

In [6]:
xldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   № п/п                     92 non-null     Int64         
 1   Дата                      170 non-null    datetime64[ns]
 2   № договора                170 non-null    string        
 3   Наименование организации  170 non-null    string        
 4   Адрес заказчика           170 non-null    string        
 5   Вид работ                 170 non-null    string        
 6   Стоимость работ           168 non-null    float64       
 7   Место хранения            170 non-null    string        
 8   Исполнитель               168 non-null    string        
 9   Основание                 169 non-null    string        
 10  Срок исполнения           162 non-null    string        
 11  Срок действия договора    28 non-null     string        
 12  № акта                

In [7]:
xldf.head()

,№ п/п,Дата,№ договора,Наименование организации,Адрес заказчика,Вид работ,Стоимость работ,Место хранения,Исполнитель,Основание,Срок исполнения,Срок действия договора,№ акта,Число,№ протокола,№ заявки
0,5,2020-01-09,03/01-20 ИК,"ГБУЗ СК ""Георгиевская РБ""",г. Георгиевск,ИК,115285.0,Папка Договора ИК,Полищук В.М,Договор,20 р.д.,<NA>,15,2020-01-14,06025-06055,359
1,6,2020-01-14,06/01-20 ИК,ООО «Стоматолог»,г. Пятигорск,ИК,95600.0,Папка Договора ИК,Косиков А.В.,Договор,20 р.д.,<NA>,17,2020-01-14,06307-06331,015
2,7,2020-01-20,П 08/01-20 ИК,"ГБУЗ СК ""ЦРБ""",г. Прохладный,ИК,25000.0,Папка Договора ИК,Панков А.Г.,Договор,15 р.д.,<NA>,42,2020-01-20,06113-06117,338
3,8,2020-01-21,П 09/01-20 ИК,"НУЗ ""ОКБ"" на ст. Минеральные Воды ОАО «РЖД»",г. Минеральные Воды,ИК,24420.0,Папка Договора ИК,Панков А.Г.,Договор,15 р.д.,<NA>,43,2020-01-21,06332-06334,035
4,9,2019-01-27,К 14/01-20 ИК,ГБУЗ РВФД МЗ РСО-А,г.Владикавказ,ИК,37303.0,Папка Договора ИК,Косиков А.В.,Договор,15 р.д.,Нет расчета,64,2020-01-27,06412-06415,047


## Обзор остальных типов

In [8]:
def get_unique_stats(df):
    return dict(zip(df.columns, df.apply(lambda x: len(x.unique()))))
    

get_unique_stats(xldf)

{'№ п/п': 78,
 'Дата': 107,
 '№ договора': 168,
 'Наименование организации': 154,
 'Адрес заказчика': 61,
 'Вид работ': 4,
 'Стоимость работ': 143,
 'Место хранения': 2,
 'Исполнитель': 12,
 'Основание': 11,
 'Срок исполнения': 21,
 'Срок действия договора': 18,
 '№ акта': 142,
 'Число': 94,
 '№ протокола': 105,
 '№ заявки': 135}

### № договора

### Наименование организации

In [9]:
# xldf['Наименование организации']=\
# xldf['Наименование организации'].apply(lambda x: re.sub(r'\s+', ' ', x))

# xldf['Наименование организации']=\
# xldf['Наименование организации'].apply(lambda x: re.sub(r'^\s+|\s+$', '', x))

### Адрес заказчика

In [10]:
xldf['Адрес заказчика'] = xldf['Адрес заказчика'].astype('string')